In [17]:

from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from typing import List
import requests
import os

from typing import Optional, List, Mapping, Any


In [14]:
api_key = os.getenv("API_KEY")

In [20]:

# Custom LLM class to wrap the API call
class MTS_Model(LLM):
    api_key: str

    def _call(self, prompt: str) -> str:
        # Make the API call with the prompt
        url = "https://api.gpt.mws.ru/v1/models"
        headers = {
            "Authorization": f"Bearer {self.api_key}"  # Replace YOUR_API_KEY with your actual API key
        }
        data = {
            "model": 'deepseek-r1-distill-qwen-32b',
            "prompt": prompt,
            "temperature": 0.2,
            "max_tokens": 300,
            "top_p": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
        }
        response = requests.post(self.api_url, json=data)
        if response.status_code == 200:
            return response.json()["text"]
        else:
            raise Exception(f"API call failed with status code {response.status_code}")
        
    @property
    def _llm_type(self) -> str:
        return "MTS_Model"


In [10]:
class MTS_Embeddings(Embeddings):
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_url = "https://api.gpt.mws.ru/v1/embeddings"
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._get_embedding(text) for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        return self._get_embedding(text)
        
    def _get_embedding(self, text: str) -> List[float]:
        headers = {'Authorization': f'Bearer {self.api_key}'}
        data = {'text': text}
        response = requests.post(self.api_url, json=data, headers=headers)
        return response.json()['data'][0]['embedding']

In [ ]:
embeddings = MTS_Embeddings(api_key)
vectorstore = FAISS.load_local("path/to/faiss/index", embeddings=embeddings) #TODO Вот это пока нету
retriever = vectorstore.as_retriever()
llm = MTS_Model(api_key=api_key)

In [ ]:
# TODO
# Тут просто кусок вайбкода, который надо переделать под задачу
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\n\nHelpful Answer:"
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)